In [1]:
import pandas as pd

In [2]:
dfs = pd.read_excel('eval_offshore_validated.xlsx', sheet_name=None)

In [3]:
dfs.keys()

dict_keys(['lance_metrics', 'lance_preds', 'clm_v0.4_aliprompt_metrics', 'clm_v0.4_aliprompt_preds', 'clm_v0.4_wishprompt_metrics', 'clm_v0.4_wishprompt_preds', 'clm_v0.5_aliprompt_metrics', 'clm_v0.5_aliprompt_preds', 'clm_v0.5_wishprompt_metrics', 'clm_v0.5_wishprompt_preds', 'emb_v0.0_metrics', 'emb_v0.0_preds', 'emb_v0.4_metrics', 'emb_v0.4_preds', 'seqclf_v0.0_metrics', 'seqclf_v0.0_preds', 'seqclf_v0.1_metrics', 'seqclf_v0.1_preds', 'seqclf_v0.2_metrics', 'seqclf_v0.2_preds', 'seqclf_v0.5_metrics', 'seqclf_v0.5_preds', 'seqclf_v0.6_metrics', 'seqclf_v0.6_preds', 'seqclf_v0.7_metrics', 'seqclf_v0.7_preds', 'seqclf_v0.8_metrics', 'seqclf_v0.8_preds', 'seqclf_v0.9_metrics', 'seqclf_v0.9_preds', 'multitask_clm_v0.1_metrics', 'multitask_clm_v0.1_preds', 'multitask_seqclf_v0.1_metrics', 'multitask_seqclf_v0.1_preds', 'multitask_emb_v0.1_metrics', 'multitask_emb_v0.1_preds'])

In [4]:
import scipy.stats as ss
import numpy as np

In [5]:
def cramer_v(x, y):
    conf_matrix = pd.crosstab(x, y)
    chi2 = ss.chi2_contingency(conf_matrix)[0]
    n = sum(conf_matrix.sum())
    phi2 = chi2/n
    r,k = conf_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))    
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min( (kcorr-1), (rcorr-1)))

In [6]:
titles = dfs['multitask_clm_v0.1_preds']['title'].tolist()

In [7]:
clms = dfs['multitask_clm_v0.1_preds']['multitask_clm_v0.1_predicted_category'].apply(lambda x: tuple(eval(x))).tolist()

In [8]:
embs = dfs['multitask_emb_v0.1_preds']['multitask_emb_v0.1_predicted_category'].apply(lambda x: tuple(eval(x))).tolist()

In [9]:
seqclfs = dfs['multitask_seqclf_v0.1_preds']['multitask_seqclf_v0.1_predicted_category'].apply(lambda x: tuple(eval(x))).tolist()

In [10]:
trus = dfs['multitask_clm_v0.1_preds']['category'].apply(lambda x: tuple(eval(x))).tolist()

In [11]:
df_all = pd.DataFrame({'clm':clms, 'emb': embs, 'seqclf': seqclfs, 'truth': trus})

In [12]:
from sklearn import metrics

In [13]:
cat2id = {}
for i in df_all.to_numpy().flatten():
    if i not in cat2id:
        cat2id[i] = len(cat2id)

In [14]:
len(cat2id)

1153

In [15]:
df_all_title = df_all.copy()
df_all_title['title'] = titles

In [16]:
df_all_id = df_all.applymap(lambda x: cat2id[x])


df_all_id['all_match'] = df_all_id[['clm', 'emb', 'seqclf']].apply(
    lambda x: len(set(x)) == 1, axis=1).tolist()

In [17]:
df_all_id['maj_vote_default_seqclf'] = df_all_id[['clm', 'emb', 'seqclf']].apply(
    lambda x: ss.mode(x, keepdims=False).mode if len(set(x)) != len(x) else x.seqclf, axis=1).tolist()

In [18]:
df_all_id.loc[:, df_all_id.columns != 'all_match'].corr(method=metrics.accuracy_score)

,clm,emb,seqclf,truth,maj_vote_default_seqclf
clm,1.000000,0.341797,0.678711,0.455078,0.717773
emb,0.341797,1.000000,0.377930,0.359375,0.416992
seqclf,0.678711,0.377930,1.000000,0.511719,0.960938
truth,0.455078,0.359375,0.511719,1.000000,0.511719
maj_vote_default_seqclf,0.717773,0.416992,0.960938,0.511719,1.000000


In [19]:
idx = df_all_id[(df_all_id.maj_vote_default_seqclf != df_all_id.seqclf) & (df_all_id.maj_vote_default_seqclf == df_all_id.truth)].index
df_all_title.loc[idx].sample(2).to_dict('records')

[{'clm': ('security & protection',
   'video surveillance',
   'surveillance cameras'),
  'emb': ('security & protection',
   'video surveillance',
   'surveillance cameras'),
  'seqclf': ('security & protection',
   'video surveillance',
   'surveillance system'),
  'truth': ('security & protection',
   'video surveillance',
   'surveillance cameras'),
  'title': '2/1pcs 2MP Analog Security Camera HD 1080P Surveillance Camera with Night Vision,Indoor Outdoor Weatherproof for Home Video Surveillance Pal System'},
 {'clm': ('jewelry & accessories', 'earrings', 'stud earrings'),
  'emb': ('jewelry & accessories', 'earrings', 'stud earrings'),
  'seqclf': ('jewelry & accessories', 'jewelry sets & more', 'body jewelry'),
  'truth': ('jewelry & accessories', 'earrings', 'stud earrings'),
  'title': '6 Pairs/lot 3-8MM Square Zircon Stud Earring Set Silver Black Color 20g Stainless Steel Ear Studs Helix Conch Tragus Piercings'}]

In [20]:
idx = df_all_id[(df_all_id.maj_vote_default_seqclf != df_all_id.seqclf) & (df_all_id.seqclf == df_all_id.truth)].index
df_all_title.loc[idx].sample(2).to_dict('records')

[{'clm': ('luggage & bags', 'coin purses & holders', 'card & id holders'),
  'emb': ('luggage & bags', 'coin purses & holders', 'card & id holders'),
  'seqclf': ('luggage & bags', "women's bags", 'wallets'),
  'truth': ('luggage & bags', "women's bags", 'wallets'),
  'title': "Credit Card Wallet for Women RFID Blocking Card Holder Bifold Leather Wallet with Zipper Pocket Women's Handbag"},
 {'clm': ('sports',
   'sports clothing',
   'shirts',
   'trainning & exercise t-shirts'),
  'emb': ('sports',
   'sports clothing',
   'shirts',
   'trainning & exercise t-shirts'),
  'seqclf': ("men's clothing", 'tops & tees', 't-shirts'),
  'truth': ("men's clothing", 'tops & tees', 't-shirts'),
  'title': "Hot sale  Ty-Son Fury Gypsy King Round Neck Fashion Sports Cotton Men's T-Shirt"}]

In [21]:
df_all_title_allmatch = df_all_title.loc[df_all_id[df_all_id.all_match].index].copy()

In [24]:
len(df_all_title_allmatch) / len(df_all_title)

0.302734375

In [22]:
df_all_title_allmatch[df_all_title_allmatch.truth.apply(lambda x: x[0]) != df_all_title_allmatch.seqclf.apply(lambda x: x[0])][
    ['title', 'seqclf', 'truth']].sample(5).to_dict('records')

[{'title': 'New Kids U-shaped Electric Toothbrush 360° Teeth Cleaner USB Rechargeable Age 2-6',
  'seqclf': ('beauty & health', 'oral hygiene', 'electric toothbrush'),
  'truth': ('mother & kids', 'baby care', 'dental care', 'toothbrushes')},
 {'title': 'Holder For Phone In Car Mobile Gravity Air Vent Monut Stand For iPhone X Xiaomi No Magnetic Smartphone Auto Smile Face Support',
  'seqclf': ('cellphones & telecommunications',
   'mobile phone accessories',
   'phone holders & stands'),
  'truth': ('automobiles & motorcycles',
   'interior accessories',
   'mounts & holder',
   'universal car bracket')},
 {'title': 'UGREEN USB C to USB 3.0 OTG Convertor Adapter On The Go Cable Free up Phone Storage',
  'seqclf': ('cellphones & telecommunications',
   'mobile phone accessories',
   'phone adapters & converters'),
  'truth': ('consumer electronics', 'accessories & parts', 'type-c adapter')},
 {'title': 'S-5XL Men Loose Long Sleeve Shirt Coat Casual Black Kurta Suit Medium Length Tops',


In [25]:
df_all_title_allmatch['allmatch_pred'] = df_all_title_allmatch.seqclf.apply(list).tolist()
df_all_title_allmatch['category'] = df_all_title_allmatch.truth.apply(list).tolist()

In [26]:
df_all_title_allmatch = df_all_title_allmatch[['title', 'allmatch_pred', 'category']]

In [27]:
from eval_from_file import perf_eval_util

In [28]:
df_all_title_allmatch['lance_pred'] = dfs['lance_preds']['lance_predicted_category'].loc[df_all_title_allmatch.index].apply(eval)

In [30]:
df_all_title['lance_pred'] = dfs['lance_preds']['lance_predicted_category'].loc[df_all_title.index].apply(eval)

In [33]:
df_all_title['category'] = df_all_title['truth']

In [35]:
perm_nonallmatch = pd.concat([perf_eval_util(df_all_title, i, 'seqclf') for i in [1,2,0,-1,-2]])
perm_nonallmatch_lance = pd.concat([perf_eval_util(df_all_title, i, 'lance_pred') for i in [1,2,0,-1,-2]])

perm_allmatch = pd.concat([perf_eval_util(df_all_title_allmatch, i, 'allmatch_pred') for i in [1,2,0,-1,-2]])
perm_allmatch_lance = pd.concat([perf_eval_util(df_all_title_allmatch, i, 'lance_pred') for i in [1,2,0,-1,-2]])

In [37]:
# multitask v0.1 full dataset
perm_nonallmatch[perm_nonallmatch['id'] == 'weighted avg']

,id,precision,recall,f1-score,support,level
29,weighted avg,0.829992,0.807617,0.803903,1024.0,1
239,weighted avg,0.723378,0.673828,0.671428,1024.0,2
781,weighted avg,0.599452,0.511719,0.526921,1024.0,0
362,weighted avg,0.711969,0.637695,0.643821,1024.0,-1
103,weighted avg,0.812740,0.772461,0.772305,1024.0,-2


In [39]:
# lance full dataset
perm_nonallmatch_lance[perm_nonallmatch_lance['id'] == 'weighted avg']

,id,precision,recall,f1-score,support,level
29,weighted avg,0.742939,0.757812,0.733032,1024.0,1
234,weighted avg,0.683832,0.647461,0.634103,1024.0,2
794,weighted avg,0.640225,0.506836,0.533479,1024.0,0
353,weighted avg,0.692509,0.623047,0.625475,1024.0,-1
101,weighted avg,0.737987,0.726562,0.712077,1024.0,-2


In [40]:
# multitask v0.1 all-match subset
perm_allmatch[perm_allmatch['id'] == 'weighted avg']

,id,precision,recall,f1-score,support,level
28,weighted avg,0.911977,0.896774,0.894799,310.0,1
147,weighted avg,0.830873,0.774194,0.780344,310.0,2
267,weighted avg,0.709391,0.677419,0.678115,310.0,0
171,weighted avg,0.810243,0.764516,0.768545,310.0,-1
75,weighted avg,0.894420,0.877419,0.875172,310.0,-2


In [41]:
# lance all-match subset
perm_allmatch_lance[perm_allmatch_lance['id'] == 'weighted avg']

,id,precision,recall,f1-score,support,level
28,weighted avg,0.827047,0.841935,0.823629,310.0,1
147,weighted avg,0.782419,0.729032,0.730545,310.0,2
286,weighted avg,0.695968,0.622581,0.636538,310.0,0
178,weighted avg,0.774892,0.725806,0.732057,310.0,-1
77,weighted avg,0.839307,0.825806,0.819636,310.0,-2
